In [1]:
!pip install colossalai

In [ ]:
import os
from pathlib import Path
import math
import colossalai
import torch
import torch.nn as nn
import torch.nn.functional as F
from colossalai.core import global_context as gpc
from colossalai.logging import get_dist_logger
from colossalai.nn import CosineAnnealingLR
from colossalai.nn.metric import Accuracy
from colossalai.trainer import Trainer, hooks
from colossalai.utils import MultiTimer, get_dataloader
from torchvision import transforms
from torchvision.datasets import MNIST
from tqdm import tqdm



class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits


config = {'BATCH_SIZE':128,'NUM_EPOCHS':30}

colossalai.launch(config=config,rank=0,world_size=1,host='127.0.0.1',port=1234)

logger = get_dist_logger()

# build 

model = LeNet5(n_classes=10)

# build dataloaders
train_dataset = MNIST(
    root=Path('./tmp/'),
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

test_dataset = MNIST(
    root=Path('./tmp/'),
    train=False,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

train_dataloader = get_dataloader(dataset=train_dataset,
                                  shuffle=True,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

test_dataloader = get_dataloader(dataset=test_dataset,
                                  add_sampler=False,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

# build criterion
criterion = torch.nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1.2e-1, momentum=0.9, weight_decay=5e-4)

# lr_scheduler
# lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[35,], gamma=0.1)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20], gamma=0.1)
# import pdb;pdb.set_trace()
engine, train_dataloader, test_dataloader, _ = colossalai.initialize(model,
                                                                      optimizer,
                                                                      criterion,
                                                                      train_dataloader,
                                                                      test_dataloader,
                                                                      )
# build a timer to measure time
timer = MultiTimer()

# create a trainer object
trainer = Trainer(
    engine=engine,
    timer=timer,
    logger=logger
)

# define the hooks to attach to the trainer
hook_list = [
    hooks.LossHook(),
    hooks.LRSchedulerHook(lr_scheduler=lr_scheduler, by_epoch=True),
    hooks.AccuracyHook(accuracy_func=Accuracy()),
    hooks.LogMetricByEpochHook(logger),
    hooks.LogMemoryByEpochHook(logger),
    hooks.LogTimingByEpochHook(timer, logger),

    # you can uncomment these lines if you wish to use them
    hooks.TensorboardHook(log_dir='./tb_logs', ranks=[0]),
    # hooks.SaveCheckpointHook(checkpoint_dir='./ckpt')
]

# start training
trainer.fit(
    train_dataloader=train_dataloader,
    epochs=gpc.config.NUM_EPOCHS,
    test_dataloader=test_dataloader,
    test_interval=1,
    hooks=hook_list,
    display_progress=True
)

[03/31/22 13:51:32] INFO     colossalai -                             ]8;id=914731;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=881752;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-03-31 13:51:32,847 INFO: Added key:                        
                             store_based_barrier_key:1 to store for                          
                             rank: 0                                                         

                    INFO     colossalai -                             ]8;id=775709;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=116084;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#252\252]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-03-31 13:51:32,855 INFO: Rank 0:                           
                             Completed store-based barrier for                               
                             key:store_based_barrier_key:1 with 1                            
                             nodes.                                                          

                    INFO     colossalai -                             ]8;id=884441;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=551011;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-03-31 13:51:32,863 INFO: Added key:                        
                             store_based_barrier_key:2 to store for                          
                             rank: 0                                                         

                    INFO     colossalai -                             ]8;id=332327;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=76456;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#252\252]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-03-31 13:51:32,868 INFO: Rank 0:                           
                             Completed store-based barrier for                               
                             key:store_based_barrier_key:2 with 1                            
                             nodes.                                                          

                    INFO     colossalai -                             ]8;id=413238;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py\distributed_c10d.py]8;;\:]8;id=435300;file:///usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py#217\217]8;;\
                             torch.distributed.distributed_c10d -                            
                             2022-03-31 13:51:32,876 INFO: Added key:                        
                             store_based_barrier_key:3 to store for                          
                             rank: 0                                                         

...

                    INFO     colossalai - colossalai - 2022-03-31 13:51:32,906  ]8;id=337621;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=418028;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: process rank 0 is bound to device 0                       

                    INFO     colossalai - colossalai - 2022-03-31 13:51:32,917  ]8;id=835769;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=20063;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#109\109]8;;\
                             INFO: initialized seed on rank 0, numpy: 1024,                  
                             python random: 1024, ParallelMode.DATA: 1024,                   
                             ParallelMode.TENSOR: 1024,the default parallel                  
                             seed is ParallelMode.DATA.                                      

                    INFO     colossalai - colossalai - 2022-03-31 13:51:32,928  ]8;id=506985;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=408344;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#113\113]8;;\
                             INFO: Distributed environment is initialized, data              
                             parallel size: 1, pipeline parallel size: 1,                    
                             tensor parallel size: 1                                         

[03/31/22 13:51:33] INFO     colossalai - colossalai - 2022-03-31 13:51:33,018  ]8;id=338916;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=869699;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#113\113]8;;\
                             INFO:                                                           
                             ========== Your Config ========                                 
                             {'BATCH_SIZE': 128, 'NUM_EPOCHS': 30}                           
                             ================================                                
                                                                                             

                    INFO     colossalai - colossalai - 2022-03-31 13:51:33,026  ]8;id=831130;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py\logger.py]8;;\:]8;id=912715;file:///usr/local/lib/python3.7/dist-packages/colossalai/logging/logger.py#113\113]8;;\
                             INFO: cuDNN benchmark = True, deterministic =                   
                             False                                                           

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs